# ISMIP6 Interactive Comparison Tool

This notebook provides interactive visualization of ISMIP6 NetCDF data using HoloViews.

Features:
- Load data directly from Google Cloud Storage
- Compare multiple models and experiments
- Linked zooming and panning across all plots
- Interactive time slider
- Colorbar synchronization

In [ ]:
# Import libraries
import numpy as np
import xarray as xr
import holoviews as hv
import hvplot.xarray
from holoviews import opts
import panel as pn
from ismip6_index import get_file_index
from grid_utils import correct_grid_coordinates

# Initialize HoloViews with bokeh backend
hv.extension('bokeh')
pn.extension()

print("Libraries loaded successfully!")

## Configuration

Specify the variable, experiments, and models to compare:

In [ ]:
# Configuration
VARIABLE = 'sftflf'  # Variable to plot
EXPERIMENTS = ['ctrl_proj_std']  # List of experiments to compare
MODELS = ['AWI/PISM1', 'UCIJPL/ISSM', 'NCAR/CISM', 'DOE/MALI']  # List of models to compare

# Optional: Select specific time step (None for all)
TIME_STEP = 0  # 0 for first time step, None to load all time steps

NAN_VALUES = [0]

print(f"Variable: {VARIABLE}")
print(f"Experiments: {EXPERIMENTS}")
print(f"Models: {MODELS}")

## Load File Index

Load the ISMIP6 file index to find the URLs for the requested data:

In [ ]:
# Load the file index
print("Loading ISMIP6 file index...")
df = get_file_index()

# Add model column
df['model'] = df['institution'] + '/' + df['model_name']

print(f"Total files in index: {len(df):,}")
print(f"Variables available: {df['variable'].nunique()}")
print(f"Models available: {df['model'].nunique()}")
print(f"Experiments available: {df['experiment'].nunique()}")

## Find Matching Files

Filter the index to find files matching our criteria:

In [ ]:
# Filter for matching files
filtered_df = df[
    (df['variable'] == VARIABLE) & 
    (df['experiment'].isin(EXPERIMENTS)) & 
    (df['model'].isin(MODELS))
]

print(f"Found {len(filtered_df)} matching files:")
print()
for _, row in filtered_df.iterrows():
    size_mb = row['size_bytes'] / (1024 * 1024)
    print(f"  {row['model']:20} | {row['experiment']:20} | {size_mb:6.1f} MB")

if len(filtered_df) == 0:
    print("\n⚠️  No files found matching the criteria!")
    print("\nTry checking available combinations:")
    for model in MODELS:
        model_df = df[(df['model'] == model) & (df['variable'] == VARIABLE)]
        if len(model_df) > 0:
            print(f"  {model}: {', '.join(model_df['experiment'].unique())}")
        else:
            print(f"  {model}: No data for variable {VARIABLE}")

## Load NetCDF Data

Load the data directly from Google Cloud Storage using xarray:

In [ ]:
# Load data from each file
datasets = {}

for _, row in filtered_df.iterrows():
    model = row['model']
    experiment = row['experiment']
    url = row['url']
    
    key = f"{model} - {experiment}"
    
    print(f"Loading {key}...")
    
    try:
        # Open dataset from GCS
        ds = xr.open_dataset(url, engine='h5netcdf')
        
        # Apply grid correction if needed
        ds = correct_grid_coordinates(ds, data_var=VARIABLE)

        # Replace specified NaN values with actual NaN
        if VARIABLE in ds:
            for nan_val in NAN_VALUES:
                ds[VARIABLE] = ds[VARIABLE].where(ds[VARIABLE] != nan_val, np.nan)
        
        # Get the variable data
        if VARIABLE in ds:
            var_data = ds[VARIABLE]
            
            # Select specific time step if requested
            if TIME_STEP is not None and 'time' in var_data.dims:
                var_data = var_data.isel(time=TIME_STEP)
            
            datasets[key] = var_data
            
            # Print info
            print(f"  Shape: {var_data.shape}")
            print(f"  Dimensions: {list(var_data.dims)}")
            print(f"  Coordinates: {list(var_data.coords)}")
            if hasattr(var_data, 'units'):
                print(f"  Units: {var_data.units}")
        else:
            print(f"  ⚠️  Variable {VARIABLE} not found in dataset")
            
    except Exception as e:
        print(f"  ❌ Error loading data: {e}")

print(f"\nLoaded {len(datasets)} datasets successfully")

## Calculate Global Value Range

For consistent colorbars across all plots:

In [ ]:
# Calculate global min/max for consistent color scale
if len(datasets) > 0:
    all_values = []
    for key, data in datasets.items():
        # Sample the data to avoid loading entire arrays
        values = data.values.flatten()
        # Remove NaN and infinite values
        valid_values = values[np.isfinite(values)]
        if len(valid_values) > 0:
            all_values.extend(valid_values)
    
    if len(all_values) > 0:
        vmin = np.percentile(all_values, 5)
        vmax = np.percentile(all_values, 95)
        print(f"Value range: {vmin:.2e} to {vmax:.2e}")
        if vmin < 0 and vmax > 0:
            abs_max = max(abs(vmin), abs(vmax))
            vmin, vmax = -abs_max, abs_max
            print(f"Adjusted to symmetric range around zero: {vmin:.2e} to {vmax:.2e}")
            cmap = 'RdBu_r'
        elif (np.abs(vmax) > np.abs(vmin)) and np.abs(vmin) < 1:
            vmin = 0
            cmap = 'blues'
        elif (np.abs(vmin) > np.abs(vmax)) and np.abs(vmax) < 1:
            vmax = 0
            cmap = 'blues_r'
        else:
            cmap = 'viridis'

    else:
        vmin, vmax = None, None
        print("No valid values found")
else:
    vmin, vmax = None, None

## Create Interactive Plots

Generate HoloViews plots with linked axes:

In [ ]:
# Create plots with linked axes
plots = []

# Find the common x and y range across all datasets to ensure linking works
all_x_coords = []
all_y_coords = []
for key, data in datasets.items():
    all_x_coords.append(data.x.values)
    all_y_coords.append(data.y.values)

# Use the first dataset's coordinates as the reference (they should all be the same grid)
x_coords = all_x_coords[0]
y_coords = all_y_coords[0]
x_range = (float(x_coords.min()), float(x_coords.max()))
y_range = (float(y_coords.min()), float(y_coords.max()))

print(f"Common x range: {x_range}")
print(f"Common y range: {y_range}")
print()

for i, (key, data) in enumerate(datasets.items()):
    print(f"Creating plot for {key}...")
    
    # Create HoloViews Image directly from the data with explicit bounds
    # This ensures all plots use the same coordinate system
    # Note: Flip the array vertically to match the coordinate system
    img = hv.Image(
        np.flipud(data.values),
        bounds=(x_range[0], y_range[0], x_range[1], y_range[1]),
        kdims=['x', 'y'],
        vdims=[data.name or 'value']
    ).opts(
        cmap=cmap,
        clim=(vmin, vmax),
        title=key,
        width=300,
        colorbar=True,
        tools=['hover', 'pan', 'wheel_zoom', 'box_zoom', 'reset'],
        xlabel='X (m)',
        ylabel='Y (m)',
        aspect='equal',
        data_aspect=1,
        fontsize={'title': 12, 'labels': 10, 'xticks': 8, 'yticks': 8}
    )
    
    plots.append(img)

print(f"\nCreated {len(plots)} plots")

## Display Linked Plots

All plots share the same x and y ranges, so zooming in one will zoom all:

In [ ]:
if len(plots) > 0:
    # Arrange plots in a grid
    if len(plots) == 1:
        layout = plots[0]
    elif len(plots) == 2:
        layout = plots[0] + plots[1]
    else:
        # Create a grid layout for more than 2 plots
        layout = hv.Layout(plots).cols(2)
    
    # Link the axes across all plots
    layout.opts(
        opts.Image(axiswise=False),  # Share axes
        opts.Layout(shared_axes=True)  # Link all axes
    )
    
    display(layout)
else:
    print("No plots to display. Please check the configuration and try again.")

## Additional Information

Display metadata about the loaded datasets:

In [ ]:
# Display detailed information about each dataset
for key, data in datasets.items():
    print(f"\n{'='*60}")
    print(f"Dataset: {key}")
    print(f"{'='*60}")
    print(f"Shape: {data.shape}")
    print(f"Dimensions: {list(data.dims)}")
    print(f"Sizes: {', '.join([f'{dim}={size}' for dim, size in data.sizes.items()])}")
    print(f"Coordinates: {list(data.coords)}")
    
    # Display attributes
    if len(data.attrs) > 0:
        print("\nAttributes:")
        for attr, value in data.attrs.items():
            print(f"  {attr}: {value}")
    
    # Display statistics
    values = data.values.flatten()
    valid_values = values[np.isfinite(values)]
    if len(valid_values) > 0:
        print("\nStatistics:")
        print(f"  Min: {np.min(valid_values):.2e}")
        print(f"  Max: {np.max(valid_values):.2e}")
        print(f"  Mean: {np.mean(valid_values):.2e}")
        print(f"  Median: {np.median(valid_values):.2e}")
        print(f"  Valid cells: {len(valid_values):,} / {len(values):,} ({100*len(valid_values)/len(values):.1f}%)")

## Tips for Interactive Exploration

- **Pan**: Click and drag to pan around
- **Zoom**: Scroll wheel to zoom in/out
- **Box Zoom**: Use the box zoom tool to select a region
- **Reset**: Click reset button to return to original view
- **Hover**: Hover over the plot to see exact values
- **Linked Views**: All plots zoom and pan together!

To change the comparison, modify the configuration in the second cell and re-run all cells.